In [1]:
import pickle

In [4]:
file = open("./actin.mod1", "rb")
data = pickle.load(file)

train = data['train']
test = data['test']

In [7]:
new_data = train + test

In [74]:
sequence = new_data[0]

In [75]:
exons = sequence['exons']

In [76]:
exon = exons[0]

In [77]:
transformers_input = []

In [78]:
transformers_input.append(f"{exon['data']}")

In [79]:
transformers_input.append(f"{exon['data']}")

In [80]:
transformers_input = []

i = 0

for sequence in new_data:
	print(i)
	i+=1
	if sequence['introns']:
		for introns in sequence['introns']:
			for intron in introns:
				transformers_input.append(f"{intron['data']} | INTRON | ACTIN | COLLETOTRICHUM")
	if sequence['exons']:
		for exons in sequence['exons']:
			for exon in exons:
				transformers_input.append(f"{exon['data']} | EXON | ACTIN | COLLETOTRICHUM")

0


TypeError: string indices must be integers, not 'str'